__UNet - perception loss in fastai v2 - Transfer learning using FMD_Basemodel__

In [ ]:
from fastai.vision.all import *
import os
import pathlib
import fastai; fastai.__version__

In [ ]:
torch.cuda.set_device(5)

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

In [ ]:
# set to toch.device("cpu"), if you do not have a graphic card to work with!
device=torch.device("cuda")

In [ ]:
path = pathlib.Path(os.getcwd())

In [ ]:
path_GT = path/'Daten/Hagen_GT'
path_noisy = path/'Daten/Hagen_noisy'
"""
use the following path for Actin data only
path_GT = path/'Daten/Hagen_actin_GT'
path_noisy = path/'Daten/Hagen__noisy'
"""

In [ ]:
# set seeds for everything for reproducibility
%run set_seeds.ipynb

In [ ]:
# run Dataloader
%run Dataloader_Hagen.ipynb

In [ ]:
# run notebook to set loss 
%run Perception_loss.ipynb

In [ ]:
# apply dataloader with batchsize of 38 for RTX4060, 16GB - loss weights may need be tuned for other bs
dls_den, dblock = get_dls(38)

In [ ]:
# setup loss function and weights for indiviual loss components - approximately equal loss in first epoch
#loss_func = FeatureLoss(vgg_m, blocks[2:5],[100], [4,11,14], [2,13,19])    # tuned weights for actin only data
loss_func = FeatureLoss(vgg_m, blocks[2:5],[100], [6,17,20], [4,27,24])    # tuned weights for all Hagen data

In [ ]:
# set metrics to track individual loss components 
metrics = LossMetrics(loss_func.metric_names)

In [ ]:
# set file name and tracking information to store training metrics
fname = 'FMD_to_Hagen_all'
cbs = CSVLogger(fname=fname+'.csv')

In [ ]:
# define model architecture and parameters, pretrained true freezes layers, enabling fine tuning
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range, pretrained=True, weights=None,
                        metrics=metrics, cbs=cbs
                        )

In [ ]:
# create denoising learner, load Baseline_FMD model, convert to fp16
learn_den = create_gen_learner()
learn_den.load('Basemodel_FMD').to_fp16()
learn_den.summary() # to check that layers are frozen

In [ ]:
# learning rate finder to set proper learning rate - high slope for loss
learn_den.lr_find()

In [ ]:
# training using the finetune function, which trains using frozen (5 epochs) and unfrozen weights (45 epochs) 
learn_den.fine_tune(95,2e-4,wd=wd,freeze_epochs=5)

In [ ]:
# save model
learn_den.save(fname)